In [11]:
import pandas as pd
import cobra
from copy import deepcopy

import sys
sys.path.append('../pycore/')
from utils import extract_details_from_rxnid

sys.path.append('../../common/')
from custom_functions import make_escher_csv

In [2]:
prods = ['hdca', 'hxdcol', 'btoh', 'tag', 'nrgn', 'polyhb', 'citm',
         'arte', 'sesqt', 'etoh', 'lac__L', 'btd', 'ibutoh', '3hppa',
         '3hppb', 'succ', 'mal__L', 'skm', 'muco', '4hbz', '4abz',
         '2phetoh', 'cou', 'rsvtol', 'styr', 'retcln', 'glyc', '13ppd']
prod_rxns = {p:[] for p in prods}

#### Compile product - active rxns dictionary
df_rxns = pd.read_excel('./input/pathways.xlsx')
for i in df_rxns.index:
    rxnid = extract_details_from_rxnid(df_rxns.id[i])[1]
    p = df_rxns.loc[i, 'product']
    
    if p == '3hpp':
        prod_rxns['3hppa'].append(rxnid)
        prod_rxns['3hppb'].append(rxnid)
    else:
        prod_rxns[p].append(rxnid)

In [3]:
model = cobra.io.load_json_model('./input/GSM_iSace1144_rba_pathwayadd.json')
model.solver = 'cplex'
model.objective = dict()

model.reactions.ATPM_c.lower_bound = 1

model.reactions.EX_glc__D_e.bounds = (-16,1000)
model.reactions.BIOMASS_AERO_SC_hvd.bounds = (0.1, 0.1)
model.reactions.EX_o2_e.bounds = (-1000,1000)
model.reactions.EX_co2_e.bounds = (-1000,1000)

# Allow YNB media uptake
model.reactions.EX_thm_e.bounds = (-1000,1000)
model.reactions.EX_ribflv_e.bounds = (-1000,1000)
model.reactions.EX_nac_e.bounds = (-1000,1000)
model.reactions.EX_pydxn_e.bounds = (-1000,1000)
model.reactions.EX_fol_e.bounds = (-1000,1000)
model.reactions.EX_pnto__R_e.bounds = (-1000,1000)
model.reactions.EX_4abz_e.bounds = (-1000,1000)
model.reactions.EX_inost_e.bounds = (-1000,1000)

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2022-08-08


In [14]:
p = 'citm'
if p in ['3hppa', '3hppb']:
    exrxn = 'EX_3hpp_e'
else:
    exrxn = 'EX_' + p + '_e'
model.reactions.get_by_id(exrxn).objective_coefficient = 1
model.reactions.THRA_c.knock_out()

for i in range(2*10, 16*10+1):
    vglc0 = 0.1 * i
    model.reactions.EX_glc__D_e.bounds = (-vglc0, 1000)
    for r in prod_rxns[p]:
        model.reactions.get_by_id(r).bounds = (0,1000)
    fba = model.optimize()
    v = fba[exrxn]
    y = fba[exrxn] * 146.1 / 180.156 / -fba['EX_glc__D_e']
    print(str(round(vglc0,1)), fba.status, round(v, 3), round(y, 3))
    
for r in prod_rxns[p]:
    model.reactions.get_by_id(r).bounds = (0,0)
model.objective = dict()

2.0 optimal 1.062 0.431
2.1 optimal 1.183 0.457
2.2 optimal 1.303 0.48
2.3 optimal 1.424 0.502
2.4 optimal 1.544 0.522
2.5 optimal 1.665 0.54
2.6 optimal 1.785 0.557
2.7 optimal 1.906 0.572
2.8 optimal 2.026 0.587
2.9 optimal 2.146 0.6
3.0 optimal 2.266 0.613
3.1 optimal 2.379 0.622
3.2 optimal 2.479 0.628
3.3 optimal 2.579 0.634
3.4 optimal 2.679 0.639
3.5 optimal 2.779 0.644
3.6 optimal 2.879 0.649
3.7 optimal 2.979 0.653
3.8 optimal 3.079 0.657
3.9 optimal 3.179 0.661
4.0 optimal 3.279 0.665
4.1 optimal 3.379 0.668
4.2 optimal 3.479 0.672
4.3 optimal 3.579 0.675
4.4 optimal 3.679 0.678
4.5 optimal 3.779 0.681
4.6 optimal 3.879 0.684
4.7 optimal 3.979 0.687
4.8 optimal 4.079 0.689
4.9 optimal 4.179 0.692
5.0 optimal 4.279 0.694
5.1 optimal 4.379 0.696
5.2 optimal 4.479 0.699
5.3 optimal 4.579 0.701
5.4 optimal 4.679 0.703
5.5 optimal 4.779 0.705
5.6 optimal 4.879 0.707
5.7 optimal 4.979 0.708
5.8 optimal 5.079 0.71
5.9 optimal 5.179 0.712
6.0 optimal 5.279 0.714
6.1 optimal 5.379 0.7

In [15]:
p = 'citm'
if p in ['3hppa', '3hppb']:
    exrxn = 'EX_3hpp_e'
else:
    exrxn = 'EX_' + p + '_e'
model.reactions.get_by_id(exrxn).objective_coefficient = 1
model.reactions.THRA_c.knock_out()

vglc0 = 10.6
model.reactions.EX_glc__D_e.bounds = (-vglc0, 1000)
for r in prod_rxns[p]:
    model.reactions.get_by_id(r).bounds = (0,1000)
fba = model.optimize()
v = fba[exrxn]
y = fba[exrxn] * 146.1 / 180.156 / -fba['EX_glc__D_e']
print(str(round(vglc0,1)), fba.status, round(v,3), round(y,3))

for r in prod_rxns[p]:
    model.reactions.get_by_id(r).bounds = (0,0)
model.objective = dict()

10.6 optimal 9.879 0.756


In [6]:
fba['EX_co2_e']

8.191312147129395

In [9]:
for rxn in model.reactions:
    eqn = rxn.reaction
    if 'co2_c' in eqn or 'co2_m' in eqn:
        v = fba[rxn.id]
        if abs(v) > 0.1:
            print(rxn.id, round(v,2), eqn)

GLYCL_m 1.66 gly_m + nad_m + thf_m <=> co2_m + mlthf_m + nadh_m + nh4_m
IGPS_c 1.27 2cpr5p_c + h_c --> 3ig3p_c + co2_c + h2o_c
ICDHy_c 0.11 icit_c + nadp_c <=> akg_c + co2_c + nadph_c
ME1_m 1.61 mal__L_m + nad_m --> co2_m + nadh_m + pyr_m
PYRDC_c 7.08 h_c + pyr_c --> acald_c + co2_c
PDH_m 1.47 coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m
HCO3E_c 5.33 co2_c + h2o_c <=> h_c + hco3_c
CO2t_c_m -4.85 co2_c <=> co2_m
CO2t_c_e 8.19 co2_c <=> co2_e


In [16]:
make_escher_csv(fba, './test.escher.csv')

In [13]:
model.reactions.ATPM_c

Reaction identifier,ATPM_c
Name,non-growth associated maintenance reaction
Memory address,0x07f79e5cbd710
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP [cytoplasm] + H2O [cytoplasm] --> ADP [cytoplasm] + H+ [cytoplasm] + phosphate [cytoplasm]
GPR,
Lower bound,1
Upper bound,1000.0


In [ ]:
GLYCL_m 1.608
IGPS_c 0.005
ICDHy_c 0
ME1_m 0
PYRDC_c 7.629
PDH_m 0
HCO3E_n 3.57
CO2t_c_m -2.0
CO2t_c_e 6.27